# PIG
- 데이터 분석인 MapReduce를 쉽게 하기위한 기능을 가진 툴
- MapReduce API를 매우 단순화시키고, SQL과 유사한 형태로 설계함
- 'pig'를 통해 실행

In [ ]:
# 전체적으로 Linux와 비슷
ls # directory 확인
mkdir demos

# pigdemo.txt 공유폴더로 옮기기
$sudo mount -t vboxsf wshare lshare # 공유폴더 연결

# 문서 복사해오기
copyFromLocal /root/lshare/pigdemo.txt demos/

# 아래 코드로 절대경로를 확인
cd demos
pwd

# 문서확인
cat pigdemo.txt

In [ ]:
# pigdemo.txt 2열 relation column으로 분할
employees = LOAD 'pigdemo.txt' AS (state, name);

# relation의 구성 확인
describe employees;

# relation 확인
DUMP employees;

# 조건에 맞게 filter하여 ca_only 저장 후 확인
ca_only = FILTER employees BY (state == 'CA');
DUMP ca_only;

# state 값이 같은 record들을 group한 relation 생성
emp_group = GROUP employees BY state;
describe emp_group;
DUMP emp_group;

# 1. emp_group txt파일로 저장
STORE emp_group INTO 'emp_group.txt';
ls # file 확인
cat emp_group.txt

# 2. emp_group csv파일로 저장
STORE emp_group INTO 'emp_group.csv' USING PigStorage(',');
hadoop fs -cat demos/emp_group.csv/part-r-00000 # 확인하기

# aliases 통해 현재 생성된 relation 확인
aliases;

In [ ]:
# PIG를 활용한 빅데이터 처리

mkdir whitehouse

# 문서 가져오기 및 확인
copyFromLocal /root/lshare/visits.txt whitehouse/
ls whitehouse

# JOIN DATABASES

In [ ]:
# congress.txt 하둡공간(whitehouse 디렉토리)에 저장 & 확인 & 출력
hadoop fs -put congress.txt whitehouse/congress.txt
hadoop fs -ls whitehouse/
hadoop fs -cat whitehouse/congress.txt

# visitors relation 생성
visitors = LOAD 'whitehouse/visits.txt' USING PigStorage(',') AS (lname:chararray, fname:chararray);

In [ ]:
# congress relation 생성
congress = LOAD 'whitehouse/congress.txt' AS (
full_title:chararray,
district:chararray,
title:chararray,
fname:chararray,
lname:chararray,
party:chararray);

# visits.txt 이름이 모두 대문자 but congress.txt는 아님
# - data 내용을 수정해주는 relation 생성
congress_data = FOREACH congress GENERATE district, UPPER(lname) AS lname, UPPER(fname) AS fname, party;

In [ ]:
# visitors relation & congress_data relation name을 join한 relation 생성
join_contact_congress = JOIN visitors BY (lname, fname), congress_data BY (lname, fname);

# joinresult 폴더에 저장
STORE join_contact_congress INTO 'joinresult';

# 하둡공간에 저장한 relation 확인
hadoop fs -cat ./joinresult/part-r-00000